In [2]:
import os
from dotenv import load_dotenv
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential


In [3]:
# Load environment variables
load_dotenv()
key = os.getenv("TEXT_ANALYTICS_KEY")
endpoint = os.getenv("TEXT_ANALYTICS_ENDPOINT")


In [4]:
# Authenticate client
credential = AzureKeyCredential(key)
client = TextAnalyticsClient(endpoint=endpoint, credential=credential)


In [9]:
# Example: Sentiment Analysis
documents = ["I love Azure Text Analytics! It's fantastic.","good lord!","azure is too hard a name","azure get the job done","azure is meh"]

try:
    response = client.analyze_sentiment(documents=documents)
    for doc in response:
        print(f"Sentiment: {doc.sentiment}")
        print(f"Confidence Scores: {doc.confidence_scores}")
except Exception as e:
    print(f"Error: {e}")

Sentiment: positive
Confidence Scores: {'positive': 1.0, 'neutral': 0.0, 'negative': 0.0}
Sentiment: positive
Confidence Scores: {'positive': 0.97, 'neutral': 0.02, 'negative': 0.01}
Sentiment: negative
Confidence Scores: {'positive': 0.0, 'neutral': 0.02, 'negative': 0.98}
Sentiment: positive
Confidence Scores: {'positive': 0.93, 'neutral': 0.07, 'negative': 0.01}
Sentiment: neutral
Confidence Scores: {'positive': 0.05, 'neutral': 0.85, 'negative': 0.1}
